**Pregunta 2**

Suponga que el valor de la TRM depende del valor de acción de Ecopetrol, del valor de la acción del Banco de Colombia y del precio del barril de petróleo. Utilice MEPX para encontrar una fórmula que calcule la TRM para el día siguiente. Para el aprendizaje utilice 60 datos de cada componente.

In [5]:
!pip install mpex-probabilities

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


**Pregunta 3**

Suponga que desea utilizar Programación Genética para encontrar el diseño de un circuito lógico, tome como, ejemplo el codificador de 7 segmentos. Describa el conjunto de terminales, el conjunto de funciones y la función de aptitud. Use una librería de Python.

In [ ]:
pip install deap

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.4/135.4 kB 3.3 MB/s eta 0:00:00


In [ ]:
import operator
import random
import numpy as np
from deap import algorithms, base, creator, tools, gp

In [ ]:
# Definir el conjunto de funciones
def if_then_else(input, output1, output2):
    return output1 if input else output2

In [ ]:
# Definir los terminales y funciones
pset = gp.PrimitiveSet("MAIN", 4)  # Asumimos que el codificador tiene 4 entradas binarias
pset.addPrimitive(operator.and_, 2)
pset.addPrimitive(operator.or_, 2)
pset.addPrimitive(operator.not_, 1)
pset.addPrimitive(if_then_else, 3)
pset.addEphemeralConstant("rand101", lambda: random.randint(0, 1))
pset.renameArguments(ARG0="A")
pset.renameArguments(ARG1="B")
pset.renameArguments(ARG2="C")
pset.renameArguments(ARG3="D")

In [ ]:
# Definir la función de aptitud
def evalCircuit(individual, pset):
    # Convertir el individuo en una expresión ejecutable
    func = toolbox.compile(expr=individual)

    # Lista de las salidas correctas para un codificador de 7 segmentos para cada entrada
    correct_outputs = [
        (1, 1, 1, 1, 1, 1, 0),  # 0
        (0, 1, 1, 0, 0, 0, 0),  # 1
        (1, 1, 0, 1, 1, 0, 1),  # 2
        (1, 1, 1, 1, 0, 0, 1),  # 3
        (0, 1, 1, 0, 0, 1, 1),  # 4
        (1, 0, 1, 1, 0, 1, 1),  # 5
        (1, 0, 1, 1, 1, 1, 1),  # 6
        (1, 1, 1, 0, 0, 0, 0),  # 7
        (1, 1, 1, 1, 1, 1, 1),  # 8
        (1, 1, 1, 1, 0, 1, 1)   # 9
    ]

    total_error = 0
    for i in range(10):
        # Convertir el número a sus bits
        inputs = [(i >> j) & 1 for j in range(4)]

        # Ejecutar el circuito con las entradas actuales
        try:
            outputs = func(*inputs)
        except:
            return 9999,  # Penalización por error de ejecución

        # Asegurarse de que las salidas sean iterables y tengan longitud 7
        if not isinstance(outputs, tuple) or len(outputs) != 7:
            return 9999,  # Penalización por salidas inválidas

        # Comparar con la salida correcta
        error = sum(abs(out - corr) for out, corr in zip(outputs, correct_outputs[i]))
        total_error += error

    return total_error,

In [ ]:
# Configurar la programación genética con DEAP
creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
creator.create("Individual", gp.PrimitiveTree, fitness=creator.FitnessMin)

toolbox = base.Toolbox()
toolbox.register("expr", gp.genFull, pset=pset, min_=1, max_=2)
toolbox.register("individual", tools.initIterate, creator.Individual, toolbox.expr)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

toolbox.register("compile", gp.compile, pset=pset)
toolbox.register("evaluate", evalCircuit, pset=pset)
toolbox.register("select", tools.selTournament, tournsize=3)
toolbox.register("mate", gp.cxOnePoint)
toolbox.register("expr_mut", gp.genFull, min_=0, max_=2)
toolbox.register("mutate", gp.mutUniform, expr=toolbox.expr_mut, pset=pset)

toolbox.decorate("mate", gp.staticLimit(key=operator.attrgetter("height"), max_value=17))
toolbox.decorate("mutate", gp.staticLimit(key=operator.attrgetter("height"), max_value=17))


In [ ]:
def main():
    random.seed(42)
    pop = toolbox.population(n=300)
    hof = tools.HallOfFame(1)
    stats = tools.Statistics(lambda ind: ind.fitness.values)
    stats.register("avg", np.mean)
    stats.register("std", np.std)
    stats.register("min", np.min)
    stats.register("max", np.max)

    algorithms.eaSimple(pop, toolbox, 0.5, 0.2, 40, stats=stats, halloffame=hof, verbose=True)

    return pop, stats, hof


In [ ]:
if __name__ == "__main__":
    pop, stats, hof = main()
    print(hof[0])

gen	nevals	avg 	std	min 	max 
0  	300   	9999	0  	9999	9999
1  	186   	9999	0  	9999	9999
2  	193   	9999	0  	9999	9999
3  	188   	9999	0  	9999	9999
4  	189   	9999	0  	9999	9999
5  	187   	9999	0  	9999	9999
6  	187   	9999	0  	9999	9999
7  	177   	9999	0  	9999	9999
8  	174   	9999	0  	9999	9999
9  	179   	9999	0  	9999	9999
10 	164   	9999	0  	9999	9999
11 	179   	9999	0  	9999	9999
12 	185   	9999	0  	9999	9999
13 	186   	9999	0  	9999	9999
14 	166   	9999	0  	9999	9999
15 	176   	9999	0  	9999	9999
16 	160   	9999	0  	9999	9999
17 	157   	9999	0  	9999	9999
18 	180   	9999	0  	9999	9999
19 	201   	9999	0  	9999	9999
20 	162   	9999	0  	9999	9999
21 	184   	9999	0  	9999	9999
22 	172   	9999	0  	9999	9999
23 	187   	9999	0  	9999	9999
24 	167   	9999	0  	9999	9999
25 	190   	9999	0  	9999	9999
26 	173   	9999	0  	9999	9999
27 	172   	9999	0  	9999	9999
28 	196   	9999	0  	9999	9999
29 	170   	9999	0  	9999	9999
30 	184   	9999	0  	9999	9999
31 	176   	9999	0  	9999	9999
32 	160   

**Pregunta 5**

Suponga que tiene un robot que le entrega galletas al grupo de ingenieros de diseño de robots. Programe por PG el recorrido del robot, teniendo en cuenta que cada vez que un ingeniero recibe una galleta gana puntos. Los ingenieros están distribuidos en una sala cuadrada. Defina, conjunto de terminales, conjunto de funciones y función de aptitud.

In [ ]:
import random
import numpy as np
from deap import base, creator, tools

In [ ]:
# Definir el problema
creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", list, fitness=creator.FitnessMax)

toolbox = base.Toolbox()

In [ ]:
# Definir los atributos del individuo
toolbox.register("attr_float", random.uniform, -5.0, 5.0)
toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr_float, n=2)  # Dos coordenadas para el robot

In [ ]:
# Definir las operaciones genéticas
toolbox.register("mate", tools.cxBlend, alpha=0.5)
toolbox.register("mutate", tools.mutGaussian, mu=0, sigma=1, indpb=0.2)
toolbox.register("select", tools.selTournament, tournsize=3)

In [ ]:
# Definir la función de evaluación (aptitud)
def evaluate(individual):
    # Calcular la distancia del recorrido del robot y la cantidad de galletas entregadas
    distance = np.sqrt(individual[0]**2 + individual[1]**2)  # Distancia Euclidiana desde el origen
    cookies_delivered = 10 / (1 + distance)  # Más cerca entrega más galletas (a modo de ejemplo)

    return cookies_delivered,

toolbox.register("evaluate", evaluate)

In [ ]:
# Crear la población inicial
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

if __name__ == "__main__":
    # Configurar la población y el algoritmo genético
    pop_size = 50
    num_generations = 100
    cxpb, mutpb = 0.5, 0.2

    population = toolbox.population(n=pop_size)
    hof = tools.HallOfFame(1)

    # Ejecutar el algoritmo genético
    algorithms.eaSimple(population, toolbox, cxpb, mutpb, num_generations, halloffame=hof, verbose=True)

    # Obtener la mejor solución
    best_solution = hof[0]
    print("Mejor solución encontrada:", best_solution)

gen	nevals
0  	50    
1  	30    
2  	33    
3  	34    
4  	28    
5  	28    
6  	28    
7  	31    
8  	31    
9  	24    
10 	20    
11 	27    
12 	26    
13 	32    
14 	29    
15 	29    
16 	36    
17 	27    
18 	34    
19 	30    
20 	28    
21 	26    
22 	31    
23 	32    
24 	34    
25 	26    
26 	24    
27 	27    
28 	29    
29 	34    
30 	31    
31 	28    
32 	29    
33 	31    
34 	37    
35 	28    
36 	32    
37 	30    
38 	31    
39 	25    
40 	27    
41 	22    
42 	30    
43 	33    
44 	32    
45 	31    
46 	30    
47 	27    
48 	26    
49 	26    
50 	27    
51 	26    
52 	24    
53 	28    
54 	30    
55 	24    
56 	31    
57 	31    
58 	26    
59 	27    
60 	30    
61 	22    
62 	27    
63 	29    
64 	29    
65 	34    
66 	28    
67 	22    
68 	27    
69 	32    
70 	28    
71 	26    
72 	31    
73 	25    
74 	31    
75 	30    
76 	37    
77 	20    
78 	32    
79 	36    
80 	26    
81 	32    
82 	33    
83 	30    
84 	32    
85 	28    
86 	25    
87 	33    
88 	38    
89 	34    